In [ ]:
!pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import itertools
import collections
import random
import numpy as np
import time 
from random import shuffle
from pyspark import SparkConf, SparkContext

In [ ]:
import os
HOME = os.getcwd()
TARGET = os.path.join(HOME, '/content/drive/My Drive/51900002_PhanThaiAn_51900157_NguyenThanhNhan_51900238_BuiThiHoaiThuong_Final/CAU2/sampledocs')

#đọc các file document 
documents = []
for article in os.listdir(TARGET):
    if article == 'stopwords':
        continue
    path = os.path.join(TARGET, article)
    with open(path, 'rU',encoding='windows-1252', errors='ignore' ) as file:
        documents.append(file.read())
#đọc stopword
stopwords = []
with open(os.path.join(TARGET, 'stopwords'), 'r') as file:
    for line in file:
        stopwords.append(line.strip())
#filter với stopword   
for i, doc in enumerate(documents):
    doc = doc.strip().replace('\n', ' ').lower()
    
    for word in stopwords:
        doc = doc.replace(' '+word+' ', ' ')
    documents[i] = doc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: 'U' mode is deprecated
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
len(documents)

103

In [ ]:
#1. Shingling
doc_shingles = []
for doc in documents:
  doc_shingles.append(set(doc.split(" ")))

In [ ]:
len(doc_shingles)

103

In [ ]:
# hàm chuyển thành ma trận 0/1
def to_binary_vector(vocab, vector):
#nếu token có trong vocab thì vị trí bằng 1 và ngược lại
  return [1 if x in vector else 0 for x in vocab]

# hàm tạo ma trận để thực hiện minhash
def create_hash():
# tạo mảng
  hashing_list = list(range(1, len(vocab) + 1))
#trộn mảng
  shuffle(hashing_list)
  return hashing_list

def build_minhash(vocab_size, niterations):
#tạo ma trận minhash
  hashes = []
  for _ in range(niterations):
    hashes.append(create_hash())
  return hashes

#hàm biến đổi thành signature_vector của
def create_sig(vector):
#mảng rỗng
  signature = []
#ma trận minhash
  minhash_func = build_minhash(len(vocab), MINHASH_COL)
#xét từng cột
  for func in minhash_func:
# xét vị trí đầu tiên có giá trị bằng 1, nếu có thì lưu giá trị tại ví trí đó vào signature vector
      for i in range(1, len(vocab)+1):
          idx = func.index(i)
          signature_val = vector[idx]
          if signature_val == 1:
              signature.append(i)
              break
  return signature

#hàm chia thành các band với độ rộng mỗi bands là r
def split_into_bands(signature_vector):
#tính r
  r = int(len(signature_vector) // BAND)
  subvecs = []
  for i in range(0, len(signature_vector), r):
    subvecs.append(signature_vector[i: i + r])
  return subvecs

#hàm tính độ tương tự jaccard
def jaccard(a, b):
  return len(a.intersection(b)) / len(a.union(b))

#hàm trả về số lượng candidate pair của 2 document
def candidate_pairs(a, b):
  a = split_into_bands(a)
  b = split_into_bands(b)
  pairs = 0
  for i in range(len(a)):
    if a[i] == b[i]:
# chỉ cần có 1 cặp band cùng hash vào 1 bucket (có giá trị bằng nhau) thì cặp được xét là cặp dự tuyển - candidate pair
      pairs = 1
      break
  return pairs

#hàm sắp xếp lại thứ tự cặp dự tuyển
def reorder_pair(x):
  if x[0][1] > x[1][1]:
    return (x[1], x[0])
  return x

#hàm tính độ tương tự và trả về kết quả nếu đạt ngưỡng
def cal_sim(x):
  SIMLARITY = 0.4
  sim = jaccard(set(x[0][0]), set(x[1][0]))
  if(sim >= SIMLARITY):
    return (sim,(x[0][1],x[1][1]))

PHẦN SONG SONG

In [ ]:
# khởi tạo spark context
conf = SparkConf().setMaster("local").setAppName("plagiarism-detector").set("spark.executor.memory", "15g")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
start = time.time()
#gom tất cả shingles lại thành 1 tập hợp
vocab = doc_shingles[0]
for i in range(1,len(doc_shingles)):
  vocab = vocab.union(doc_shingles[i])
  
SIMLARITY = 0.4
# tạo RDD đầu tiên để lưu trữ các shingle
first_rdd = sc.parallelize(doc_shingles)
first_rdd.cache()

# tạo RDD thứ 2 để biến đổi các shingles thành binary vector
second_rdd = first_rdd.map(lambda x : to_binary_vector(vocab, x))
second_rdd.cache()

# tạo một hằng số bằng nửa độ dài của vocab lưu số cột của bước MinHashing
MINHASH_COL = len(vocab)//2

# tạo RDD thứ 3 để hash các vector nhị phân thành signature vector - MinHashing
third_rdd = second_rdd.map(lambda x : create_sig(x))
third_rdd.cache()

# tạo một hằng số lưu số lượng band bằng nửa độ dài của signature vector
BAND = len(vocab)//4

# lưu với chỉ số tượng chưng cho từng document
third_rdd =  third_rdd.zipWithIndex()
third_rdd.cache()

#tạo tích Đề cát, ghép cặp các vector với nhau
third_rdd = third_rdd.cartesian(third_rdd)
third_rdd.cache()

#loại đi các cặp trùng vd: ((...),0), ((...),0)
third_rdd = third_rdd.filter(lambda x: x[0] != x[1])
third_rdd.cache()

#chuyển đổi thứ tự các cặp vd: [(1, 2), (2, 1)] thành [(1, 2), (1, 2)]
third_rdd = third_rdd.map(lambda x: reorder_pair(x))
third_rdd.cache()

#lấy các cặp dự tuyển có ít nhất 1 cặp band được hash vào cùng 1 bucket
third_rdd = third_rdd.filter(lambda x: (candidate_pairs(x[0][0],x[1][0])) >= 1)
third_rdd.cache()

#tính khoảng cách các cặp dự tuyển
third_rdd = third_rdd.map(lambda x: cal_sim(x))
third_rdd.cache()

#do hàm tính khoảng cách có xét điều kiện kết quả tối thiểu để trả về, nên sẽ có vài vị trí có giá trị bằng None. Tiến hành filter bỏ None
third_rdd = third_rdd.filter(lambda x: x != None)
third_rdd.cache()

#bỏ các giá trị trùng lăp
third_rdd = third_rdd.distinct()
third_rdd.cache()

#in kết quả
result = third_rdd.collect()
stop = time.time()
print("Excute time: ", stop - start)
for row in result:
  print("Distance", row[1][0]," - ",row[1][1],": Similarity: ", row[0])

Excute time:  815.6908321380615
Distance 0  -  1 : Similarity:  0.49382716049382713
Distance 0  -  2 : Similarity:  0.6707317073170732
Distance 0  -  3 : Similarity:  0.7333333333333333
Distance 0  -  6 : Similarity:  0.5405405405405406
Distance 0  -  14 : Similarity:  0.49390243902439024
Distance 0  -  15 : Similarity:  0.6290322580645161
Distance 0  -  16 : Similarity:  0.4426229508196721
Distance 0  -  18 : Similarity:  0.45251396648044695
Distance 0  -  31 : Similarity:  0.4550561797752809
Distance 0  -  36 : Similarity:  0.49390243902439024
Distance 0  -  37 : Similarity:  0.49390243902439024
Distance 0  -  38 : Similarity:  0.6171875
Distance 0  -  41 : Similarity:  0.44751381215469616
Distance 0  -  43 : Similarity:  0.50625
Distance 0  -  45 : Similarity:  0.46285714285714286
Distance 0  -  62 : Similarity:  0.65
Distance 0  -  64 : Similarity:  0.4682080924855491
Distance 0  -  65 : Similarity:  0.7291666666666666
Distance 0  -  66 : Similarity:  0.5
Distance 0  -  68 : Simila

PHẦN TUẦN TỰ

In [ ]:
start = time.time()
#gom tất cả shingles lại thành 1 tập hợp
vocab = doc_shingles[0]
for i in range(1,len(doc_shingles)):
  vocab = vocab.union(doc_shingles[i])

# tạo một hằng số bằng nửa độ dài của vocab lưu số cột của bước MinHashing
MINHASH_COL = len(vocab)//2
# tạo một hằng số lưu số lượng band bằng nửa độ dài của signature vector
BAND = len(vocab)//4

binaries = []
for doc in doc_shingles:

# biến đổi các shingles thành binary vector
  bi = to_binary_vector(vocab, doc)
  binaries.append(bi)

# hash các vector nhị phân thành signature vector - MinHashing
sigs_vec = []
sigs_splited_vec = []
for bi in binaries:
#lưu các vector chữ ký
  sig = create_sig(bi)
  sigs_vec.append(sig)
# chia thành các band, với số lượng band bằng nửa độ dài của signature vector thì mỗi band sẽ có kích thước r bằng 2
  sig2 = split_into_bands(sig)
  sigs_splited_vec.append(sig2)

#chia thành các cặp 1 - 1
pairs = itertools.combinations(sigs_splited_vec,2)
cand_pairs = []
for pair in pairs:
# lưu chỉ số các cặp cùng với khả năng candidate của chúng
  cand_pairs.append((candidate_pairs(pair[0], pair[1]), (sigs_splited_vec.index(pair[0]), sigs_splited_vec.index(pair[1]))))

# lấy về các candidate pair có ít nhất 1 cặp band cùng hash vào 1 bucket
selected_pairs = []
for c_pair in cand_pairs:
  if(c_pair[0] >= 1):
    selected_pairs.append(c_pair[1])

#ngưỡng độ tương tự
SIMLARITY = 0.4

# tính độ tương tự và so sánh với ngưỡng
for pair in selected_pairs:
  sim = jaccard(set(sigs_vec[pair[0]]), set(sigs_vec[pair[1]]))
  if(sim >= SIMLARITY):
    print("Distance", pair[0]," - ",pair[1],": Similarity: ", sim)

stop = time.time()
print("Excute time: ", stop - start)


Distance 0  -  1 : Similarity:  0.5135135135135135
Distance 0  -  2 : Similarity:  0.7162162162162162
Distance 0  -  4 : Similarity:  0.8048780487804879
Distance 0  -  6 : Similarity:  0.5174825174825175
Distance 0  -  10 : Similarity:  0.4868421052631579
Distance 0  -  14 : Similarity:  0.47435897435897434
Distance 0  -  20 : Similarity:  0.4567901234567901
Distance 0  -  36 : Similarity:  0.45962732919254656
Distance 0  -  37 : Similarity:  0.44785276073619634
Distance 0  -  41 : Similarity:  0.45121951219512196
Distance 0  -  42 : Similarity:  0.4111111111111111
Distance 0  -  45 : Similarity:  0.4451219512195122
Distance 0  -  47 : Similarity:  0.4431137724550898
Distance 0  -  59 : Similarity:  0.40437158469945356
Distance 0  -  62 : Similarity:  0.6186440677966102
Distance 0  -  64 : Similarity:  0.43529411764705883
Distance 0  -  65 : Similarity:  0.7157894736842105
Distance 0  -  66 : Similarity:  0.4506172839506173
Distance 0  -  72 : Similarity:  0.5104895104895105
Distance 0